In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

In [ ]:
## parameters cell
itp_id = 293

In [ ]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [ ]:
# airtable_organizations

In [ ]:
if itp_id in [61, 301]:
    analysis_date = dt.date(2022, 6, 8) ##wednesday, new tables for Napa operators
elif itp_id == 273:
    analysis_date = dt.date(2022, 2, 8) ## override for SacRT
elif itp_id == 208:
    analysis_date = dt.date(2022, 7, 20) ## override for MST
elif itp_id == 282:
    analysis_date = dt.date(2022, 9, 7) ## override for SFMTA
else:
    analysis_date = dt.date(2022, 6, 1) #rebuild site

In [ ]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d (%A)')
human_date

In [ ]:
analysis_date = dt.date(2022, 9, 7)

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
agency_name = rt_day.calitp_agency_name
if itp_id == 61:
    district = '04 - Oakland' ## Airtable error for 61
else:
    district = airtable_organizations.caltrans_district.iloc[0]
agency_name, district

# Santa Barbara Metropolitan Transit District

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for Santa Barbara Metropolitan Transit District, with a map each for the morning peak, midday, and afternoon peak periods on June 01 (Wednesday).
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

In [ ]:
rt_day.rt_trips.route_short_name.unique()

In [ ]:
fwy_rts = ['12X', '24X', '15X']

In [ ]:
local_rts = [rt for rt in rt_day.rt_trips.route_short_name.unique() if rt not in fwy_rts]

In [ ]:
import branca
TWENTY_65_COLORSCALE = branca.colormap.step.RdYlGn_10.scale(vmin=20, vmax=65)

## Morning Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00', route_names=fwy_rts)

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', colorscale=TWENTY_65_COLORSCALE)

In [ ]:
m

In [ ]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00', route_names=local_rts)

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()

## Midday

In [ ]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00', route_names=fwy_rts)

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', colorscale=TWENTY_65_COLORSCALE)

In [ ]:
m

In [ ]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00', route_names=local_rts)

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()

## Afternoon Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=fwy_rts)

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', colorscale=TWENTY_65_COLORSCALE)

In [ ]:
m

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=local_rts)

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()